In [1]:
from pathlib import Path

import automech
from project_utilities import p_, util, workflow
from protomech import mess

root_path = Path("..")
tag = "Z_combined_v0"

mech0 = automech.io.read(p_.stereo_mechanism(tag, "json", p_.data(root_path)))

In [2]:
mech0, _ = automech.replace_unstable_products(mech0)

/home/avcopan/proj/project-cyclopentane/code/protomech/src/automech/_mech.py:1279: UserWarning: This function ignores stereochemistry.
  warnings.warn("This function ignores stereochemistry.", stacklevel=1)


In [3]:
automech.resonant_unstable_species_names(mech0)

['S(2258)z',
 'S(2258)e',
 'C5O2nexammzzz',
 'C5O2nexammzze',
 'C5O2nexammzez',
 'C5O2nexammzee',
 'C5O2nexammezz',
 'C5O2nexammeze',
 'C5O2nexammeez',
 'C5O2nexammeee',
 'C5O2jrscln']

In [4]:
import polars as pl
from automech.reaction import Reaction

instab_names = automech.resonant_unstable_species_names(mech0)
mech0.reactions.filter(
    # pl.col(Reaction.products).list.set_intersection(instab_names).list.len() > 0
    pl.col(Reaction.reactants).list.set_intersection(instab_names).list.len() > 0
)

reactants,products,formula,orig_reactants,orig_products,reversible,rate,amchi,canon,pes,subpes,channel,replacements
list[str],list[str],struct[18],list[str],list[str],bool,struct[18],str,bool,i64,i64,i64,struct[3]
"[""S(2258)e""]","[""C3H4O(165)"", ""OH(4)""]","{5,null,null,null,null,3,null,2,null,null,null,null,null,null,null,null,null,null}","[""S(2258)""]","[""C3H4O(165)"", ""OH(4)""]",null,null,"""AMChI=1/C3H5O2/c1-2-3-5-4/h2-4…",true,1,1,1,"{[],[],[]}"
"[""S(2258)z""]","[""C3H4O(165)"", ""OH(4)""]","{5,null,null,null,null,3,null,2,null,null,null,null,null,null,null,null,null,null}","[""S(2258)""]","[""C3H4O(165)"", ""OH(4)""]",null,null,"""AMChI=1/C3H5O2/c1-2-3-5-4/h2-4…",true,1,2,2,"{[],[],[]}"
"[""C5O2jrscln""]","[""CPND2(626)"", ""OH(4)""]","{7,null,null,null,null,5,null,2,null,null,null,null,null,null,null,null,null,null}","[""C5O2jrscln""]","[""CPND2(626)"", ""OH(4)""]",null,null,"""AMChI=1/C5H7O2/c6-7-5-2-1-3-4-…",true,3,1,19,"{[],[],[]}"
"[""C5O2nexammzze""]","[""C5H6O(645)z"", ""OH(4)""]","{7,null,null,null,null,5,null,2,null,null,null,null,null,null,null,null,null,null}","[""C5O2nexamm""]","[""C5H6O(645)"", ""OH(4)""]",null,null,"""AMChI=1/C5H7O2/c1-2-3-4-5-7-6/…",true,3,1,20,"{[],[],[]}"
"[""C5O2nexammeee""]","[""C5H6O(645)e"", ""OH(4)""]","{7,null,null,null,null,5,null,2,null,null,null,null,null,null,null,null,null,null}","[""C5O2nexamm""]","[""C5H6O(645)"", ""OH(4)""]",null,null,"""AMChI=1/C5H7O2/c1-2-3-4-5-7-6/…",true,3,2,43,"{[],[],[]}"
…,…,…,…,…,…,…,…,…,…,…,…,…
"[""C5O2nexammeze""]","[""C5H6O(645)z"", ""OH(4)""]","{7,null,null,null,null,5,null,2,null,null,null,null,null,null,null,null,null,null}","[""C5O2nexamm""]","[""C5H6O(645)"", ""OH(4)""]",null,null,"""AMChI=1/C5H7O2/c1-2-3-4-5-7-6/…",true,3,4,45,"{[],[],[]}"
"[""C5O2nexammezz""]","[""C5H6O(645)z"", ""OH(4)""]","{7,null,null,null,null,5,null,2,null,null,null,null,null,null,null,null,null,null}","[""C5O2nexamm""]","[""C5H6O(645)"", ""OH(4)""]",null,null,"""AMChI=1/C5H7O2/c1-2-3-4-5-7-6/…",true,3,5,46,"{[],[],[]}"
"[""C5O2nexammzee""]","[""C5H6O(645)e"", ""OH(4)""]","{7,null,null,null,null,5,null,2,null,null,null,null,null,null,null,null,null,null}","[""C5O2nexamm""]","[""C5H6O(645)"", ""OH(4)""]",null,null,"""AMChI=1/C5H7O2/c1-2-3-4-5-7-6/…",true,3,6,47,"{[],[],[]}"


In [5]:
mess_source_path = p_.mess_source(root_path)
mess_inp_files = list(mess_source_path.glob("*/mess.inp"))
surfs = [mess.surf.from_mess_input(f) for f in mess_inp_files]
surf = mess.surf.combine(surfs)
surf = mess.surf.merge_resonant_instabilities(surf, mech0)
print(mess_inp_files)
# mess.net.display(surf, mech=mech0)

[PosixPath('../data/mess/source/04_1-2/mess.inp'), PosixPath('../data/mess/source/06_37/mess.inp'), PosixPath('../data/mess/source/05_1-14/mess.inp'), PosixPath('../data/mess/source/03_50-51/mess.inp'), PosixPath('../data/mess/source/06_35/mess.inp'), PosixPath('../data/mess/source/02_05/mess.inp'), PosixPath('../data/mess/source/03_46/mess.inp'), PosixPath('../data/mess/source/05_15/mess.inp'), PosixPath('../data/mess/source/01_01/mess.inp'), PosixPath('../data/mess/source/03_49/mess.inp'), PosixPath('../data/mess/source/07_01/mess.inp'), PosixPath('../data/mess/source/05_16/mess.inp'), PosixPath('../data/mess/source/03_48/mess.inp'), PosixPath('../data/mess/source/05_17/mess.inp'), PosixPath('../data/mess/source/02_1-4/mess.inp'), PosixPath('../data/mess/source/03_1-42/mess.inp'), PosixPath('../data/mess/source/03_54/mess.inp'), PosixPath('../data/mess/source/03_44/mess.inp'), PosixPath('../data/mess/source/06_1-34/mess.inp'), PosixPath('../data/mess/source/08_01/mess.inp'), PosixPat

In [6]:
mech = automech.merge_resonant_instabilities(mech0, remove=True)

In [7]:
# Print reagents that are not on the MESS surface
for rgt_names in automech.reaction.reagents(mech.reactions):
    node = next((n for n in surf.nodes if n.names_list == rgt_names), None)
    if node is None:
        print(rgt_names)

In [8]:
surf_dct = mess.surf.split_stoichiometries(surf, mech)
print(surf_dct)

{'C5H9': Surface(nodes=[UnimolNode(energy=0.0, fake=False, mess_body='  WellExtensionCap[kcal/mol]    44.80\n  Species\n    RRHO\n      Geometry[angstrom]        14\n        C         -0.00020       1.27243      -0.00008\n        C         -1.23047       0.42891       0.12695\n        H         -0.00033       2.35319      -0.00005\n        C         -0.73007      -0.98151      -0.23575\n        H         -1.61439       0.44166       1.15806\n        H         -2.05319       0.76678      -0.51072\n        C          0.73031      -0.98136       0.23570\n        H         -0.76346      -1.11142      -1.32171\n        H         -1.32568      -1.77731       0.21444\n        C          1.23040       0.42920      -0.12686\n        H          0.76372      -1.11140       1.32165\n        H          1.32614      -1.77696      -0.21456\n        H          2.05287       0.76726       0.51101\n        H          1.61449       0.44217      -1.15790\n      Core RigidRotor\n        SymmetryFactor     

In [9]:
mess_calc_path = p_.mess_calc(root_path)
mess_calc_path.mkdir(exist_ok=True)

for stoich, stoich_surf in surf_dct.items():
    stoich_path = mess_calc_path / stoich
    stoich_path.mkdir(exist_ok=True)
    mess_inp_file = stoich_path / "mess.inp"
    mess_inp_file.write_text(mess.surf.mess_input(stoich_surf))